In [1]:
import pandas as pd

FOREST = '/kaggle/input/global-forest-insights-1990-2020-fao/Forest_Area.csv'

df = pd.read_csv(filepath_or_buffer=FOREST)
# df['Forest Area, 2015'] = df['Forest Area, 2015'].astype(float)
df.head()

,CountryID,Country and Area,"Forest Area, 1990","Forest Area, 2000","Forest Area, 2010","Forest Area, 2015","Forest Area, 2020","Total Land Area, 2020","Forest Area as a Proportion of Total Land Area, 2020","Deforestation, 2015-2020","Total Forest Area Affected by Fire, 2015"
0,NaN,WORLD,4236433.42,4158049.52,4106316.94,,4058930.81,13003000.0,31.13,10200,98040
1,4.0,Afghanistan,1208.44,1208.44,1208.44,1208.44,1208.44,65286.0,1.85,...,...
2,8.0,Albania,788.80,769.30,782.07,789.19,788.90,2740.0,28.79,...,...
3,12.0,Algeria,1667.00,1579.00,1918.00,1956,1949.00,238174.0,0.82,5.4,...
4,16.0,American Samoa,18.07,17.73,17.43,17.28,17.13,20.0,85.65,...,0


Our 2015 data is really wonky and we need to clean it up.

In [2]:
import numpy as np

def doclean(arg) -> float:
    if isinstance(arg, str):
        if arg.isnumeric():
            return float(arg)
        else:
            return np.nan
    return np.nan

df['Forest Area, 2015'] = df['Forest Area, 2015'].apply(func=doclean)

Now we can add the ISO-3 codes so we can visualize our data.

In [3]:
iso_df = pd.read_csv(filepath_or_buffer='/kaggle/input/iso-country-codes-global/wikipedia-iso-country-codes.csv', usecols=['Alpha-3 code', 'Numeric code'])
# if we rename this column as part of setting the type to float we can use the same name in both frames in the merge below
iso_df['CountryID'] = iso_df['Numeric code'].astype(float)
forest_df = df.merge(right=iso_df, on='CountryID', how='inner')
forest_df.head()

,CountryID,Country and Area,"Forest Area, 1990","Forest Area, 2000","Forest Area, 2010","Forest Area, 2015","Forest Area, 2020","Total Land Area, 2020","Forest Area as a Proportion of Total Land Area, 2020","Deforestation, 2015-2020","Total Forest Area Affected by Fire, 2015",Alpha-3 code,Numeric code
0,4.0,Afghanistan,1208.44,1208.44,1208.44,NaN,1208.44,65286.0,1.85,...,...,AFG,4
1,8.0,Albania,788.80,769.30,782.07,NaN,788.90,2740.0,28.79,...,...,ALB,8
2,12.0,Algeria,1667.00,1579.00,1918.00,1956.0,1949.00,238174.0,0.82,5.4,...,DZA,12
3,16.0,American Samoa,18.07,17.73,17.43,NaN,17.13,20.0,85.65,...,0,ASM,16
4,20.0,Andorra,16.00,16.00,16.00,16.0,16.00,47.0,34.04,...,...,AND,20


We don't expect forest areas to change enough in ten years to be visible in a choropleth but we should look anyway.

In [4]:
from plotly import express

columns = ['Forest Area, 1990','Forest Area, 2000', 'Forest Area, 2010', 'Forest Area, 2020', ]

for color in columns:
    express.choropleth(data_frame=forest_df, locations='Alpha-3 code', color=color, title=color,).show()

The forest area as a percentage of total land area is more interesting, but it mostly tells us that the planet has both hot and cold deserts.

In [5]:
express.choropleth(data_frame=forest_df, locations='Alpha-3 code', color='Forest Area as a Proportion of Total Land Area, 2020',
                   title='Forest Area as a Proportion of Total Land Area, 2020',).show()